# Convolutional Layers

- toc: true
- badges: true
- comments: false
- categories: [jax, convolution, pooling]
- hide: true

## Introduction

In this post, I'll start by implementing a basic convolutional layer using numpy and validate it against Keras.  After this, I'll write a more efficient one using JAX.

## Import Libraries

For now, I only need numpy and tensorflow.

In [18]:
import numpy as np
import tensorflow as tf

Here's a small sequential model consisting of a convolutional layer and max-pooling layer.

In [33]:
layer = tf.keras.layers.Conv2D(filters=4, kernel_size=(2, 2), strides=(2,2), padding='valid')

For this experiment, I don't care what the input to the model is.  So I'll just create a 1-element batch consisting of a random 28-by-28 array and apply `model` to it.  

In [35]:
inputs = np.random.randn(1,28,28,3)
outputs = layer(inputs)

2022-07-30 15:14:21.032488: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/bin:/usr/local/lib:
2022-07-30 15:14:21.032522: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-30 15:14:21.032810: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2022-07-30 15:14:21.041129: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Let's check the shape of `inputs` and `outputs`.  

In [36]:
print(f'Feature Mapping:  {inputs.shape} -> {outputs.shape}')

Feature Mapping:  (1, 28, 28, 3) -> (1, 14, 14, 4)


The `outputs` is a 1-element batch consisting of a 7-by-7 array with 4 channels.

Getting the pooling layer output is easy; it's the same as `outputs`.  However, it's not immediately obvious how to get the intermediate convolution output. The recommended way to extract the outputs from all the layers in your model is to use the so-called *Functional* API.  

## Convolutional Layer

This function filters a single image with every output filter and adds the bias term, resulting in a rank 3 array.  The first two levels of the nested loop extract a rank 3 chunk from `image`, while the third level of the nested loop performs the filtering and biasing.  After a chunk is processed and the results placed in the output array `y`, the filter shape and stride is used to calculate the next chunk position.  

In [88]:
def filter_image(image, filters, biases, strides):
    
    xm, xn, _  = image.shape 
    km, kn, *_ = filters.shape 
    
    num_output_channels = len(biases)
    
    sm, sn = strides
    ym, yn = 1 + ((xm - km + 1)//sm), 1 + ((xn - kn + 1)//sn)
    
    y = np.zeros((ym, yn, num_output_channels))

    for iy, ix in enumerate(range(0, xm-km+1, sm)):
        for jy, jx in enumerate(range(0, xn-kn+1, sn)):
            # Apply each output filter and bias term to this chunk
            chunk = image[ix:ix+km,jx:jx+kn,:]
            for channel in range(num_output_channels):
                y[iy,jy,channel] = np.sum(filters[...,channel] * chunk) + biases[channel]
            
    return y

Once we have an algorithm to filter a single image, it's very simple to filter a batch of images.  Here's the code:

In [148]:
def filter_image_batch(images, filters, biases, strides):
    return np.array([filter_image(image, filters, biases, strides) for image in images])

This function requires a little explanation.  First the list comprehension passed to the `np.array` function, i.e.

```python
[filter_image(image, filters, biases, strides) for image in images]
```
applies `filter_image` to each image in the batch, resulting in a list of filtered images.  By passing this list to the `np.array` function, it's converted to an `ndarray` with a leading batch dimension.
    

In [125]:
images = np.random.randn(2,28,28,3)

In [135]:
keras_layer = tf.keras.layers.Conv2D(filters=4, kernel_size=(2, 2), strides=(2,2), padding='valid')
#keras_output = keras_layer(x)

In [127]:
%%capture
image = np.random.randn(1,28,28,3)
keras_output = layer(image)[0,:,:,:]
filters, biases = layer.get_weights()
strides = (2,2)

filter_image(np.squeeze(image), filters, biases, strides)

In [121]:
%%capture
keras_output

Now I can make a little class that contains the filters, biases, and other necessary parameters for a convolutional layer.  The `__call__` method correlates each output filter with the input image and adds the bias.  Each filtered output is added to a list, and converted to a single-element batch.

In [145]:
keras_layer.get_weights()[0].shape

(2, 2, 3, 4)

In [144]:
keras_layer.build(input_shape=(28,28,3))

In [130]:
my_layer = MyConv2D.setup(keras_layer)
my_output = my_layer(images)

In [131]:
my_output.shape


(2, 14, 14, 4)

## Compare to Keras

To compare my convolutional layer to Keras', I need to get any convolutional-relevant parameters from the Keras model and use them to initialize my layer.

In [42]:
kernels, biases = model.layers[0].get_weights()
strides = model.layers[0].strides
keras_conv_features = keras_features[0]

NameError: name 'model' is not defined

In [290]:
my_layer = MyConv2D(kernels, biases, strides=strides)
my_conv_features = my_layer(inputs)

In [292]:
assert np.all(np.isclose(keras_conv_features, my_conv_features, atol=1e-6))
assert np.all(np.isclose(jax_result, my_conv_features, atol=1e-6))

### Convolution in JAX

In [ ]:
jax_result = jax.lax.conv_general_dilated(
    lhs=inputs,
    rhs=kernels,
    window_strides=strides,
    padding='valid',
    dimension_numbers=('NHWC', 'HWIO', 'NHWC')
)